## 💼 "All-Weather" Risk-Parity Portfolio Construction

### ETF Selection and Data Download

Importing required packages

In [49]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

Initialization of portfolio parameters

In [48]:
TICKERS = ['SPY', 'TLT', 'GLD', 'TIP', 'BIL']  # Equities, Fixed Income, Commodities, Inflation-protected, Cash
START_DATE = '2015-01-01'
END_DATE = '2025-06-27'
VOL_LOOKBACK = 126  # Volatility Lookback Period Length

Function for fetching daily price data

In [45]:
def get_etf_data(tickers: list[str], start: str, end: str):
    """
    Fetches financial data from Yahoo Finance API for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
        pd.DataFrame: DataFrame with date-indexed closing price data
    """
    
    data = yf.download(tickers, start, end, auto_adjust=True)['Close']
    return data.dropna()

### Volatility and Weight Calculation
Functions for calculating rolling volatility and daily risk-parity weights

In [46]:
def calculate_rolling_volatility(tickers: list[str], start: str, end: str, window: int):
    """
    Calculates rolling 126-day volatility for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
        window (int): Length of lookback period in number of days

    Returns:
        pd.DataFrame: DataFrame with date-indexed rolling 126-day volatility
    """

    # Fetch price data
    data = get_etf_data(tickers, start, end)

    # Compute daily return
    returns = data.pct_change()

    # Compute rolling volatility
    rolling_vol = returns.rolling(window=window).std()

    return rolling_vol

In [47]:
def calculate_risk_parity_weights(tickers: list[str] = TICKERS, start: str = START_DATE, end: str = END_DATE, window: int = VOL_LOOKBACK):
    """
    Calculates daily risk-parity weights for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
        window (int): Length of lookback period in number of days

    Returns:
        pd.DataFrame: DataFrame with date-indexed risk-parity weights
    """

    # Fetch rolling volatility
    vol = calculate_rolling_volatility(tickers, start, end, window)

    # Invert volatility
    inv_vol = 1 / vol

    # Normalize rows so weights sum to 1
    weights = inv_vol.div(inv_vol.sum(axis=1), axis=0)

    return weights.dropna()

### Backtesting

In [ ]:
# def backtest_portfolio(weights, returns)

# def stress_test_period(portfolio_returns, start, end)

# def compute_var_cvar(returns, alpha=0.05)

### Performance Metrics and Plots